In [15]:
from dataclasses import dataclass
from pathlib import Path
import torch
import torch.nn as nn
from jaxtyping import Float, Int
import requests
import unicodedata
from collections import Counter


# Training Data

In [2]:
def get_gutenberg_book(
	id: int|None = 84,
	data_temp: Path|str = "../../../../data/gutenberg_data",
	remove_gutenberg_meta: bool = True,
) -> str:
	
	data_temp = Path(data_temp)
	data_temp.mkdir(parents=True, exist_ok=True)
	
	url: str = f"https://www.gutenberg.org/cache/epub/{id}/pg{id}.txt"
	data_path: Path = Path(data_temp) / f"{id}.txt"
	data: str
	# read from cache if it exists
	if data_path.exists():
		with open(data_path, 'r', encoding='utf-8') as file:
			data = file.read()
	else:
		# download if it doesn't exist
		response = requests.get(url)
		response.raise_for_status()  # Ensure that the download was successful
		data = response.text

		# save to cache
		with open(data_path, 'w', encoding='utf-8') as file:
			file.write(data)

	# remove header/footer
	if remove_gutenberg_meta:
		data = '***'.join(data.split('***')[2:])
		data = '***'.join(data.split('***')[:-1])
	
	return data

def get_many_books(
		ids: list[int],
		data_temp: Path|str = "../data/gutenberg_data",
	) -> list[list[str]]:
	
	data: list[str] = []
	for id in ids:
		print(f"Getting book {id}...")
		item: str = get_gutenberg_book(id, data_temp)
		print(f"\t{len(item)} characters read")
		data.append(item)
	
	return data

# Model Definition

In [3]:
@dataclass
class Config():
    d_model: int
    d_vocab: int
    d_hidden: int

In [4]:
class MLP(nn.Module):
    def __init__(self, config: Config):
        super().__init__()
        self.linear1 = nn.Linear(config.d_model, config.d_hidden)
        self.linear2 = nn.Linear(config.d_hidden, config.d_model)
        

    def forward(self, x: Float[torch.Tensor, "seq_len d_model"]) -> Float[torch.Tensor, "seq_len d_model"]:
        x = self.linear1(x)
        x = torch.relu(x)
        x = self.linear2(x)
        return x

In [5]:
class AttentionHead(nn.Module):
    def __init__(self, config: Config):
        super().__init__()
        self.config = config
        self.W_qk = nn.Linear(config.d_model, config.d_model)
        self.W_vo = nn.Linear(config.d_model, config.d_model)
        self.softmax = nn.Softmax(dim=-1)
        

    def create_mask(self, n_c: int) -> torch.Tensor:
        mask: Float[torch.Tensor, "seq_len seq_len"] = torch.triu(-1 * torch.inf * torch.ones(n_c, n_c), diagonal=1)
        return mask

    def forward(self, x: Float[torch.Tensor, "seq_len d_model"]) -> Float[torch.Tensor, "seq_len d_model"]:
        #create mask, with size n_c x n_c
        mask = self.create_mask(x.shape[0])

        #compute attention scores
        # A = softmax((X @ W_qk @ X^T) + M) @ X @ W_vo
        A = self.softmax((self.W_qk(x)) @ x.transpose(0, -1) + mask) @ self.W_vo(x)
        return A

In [6]:
class TransformerBlock(torch.nn.Module):
    def __init__(self, config: Config):
        super().__init__()
        self.attention_head = AttentionHead(config)
        self.mlp = MLP(config)

    def forward(self, x: Float[torch.Tensor, "seq_len d_model"]) -> Float[torch.Tensor, "seq_len d_model"]:
        return x + self.attention_head(x) + self.mlp(x)

In [7]:
class Transformer(torch.nn.Module):
    def __init__(self, num_blocks: int, config: Config):
        super().__init__()
        self.config = config
        self.embedding = nn.Linear(config.d_vocab, config.d_model)
        self.blocks = nn.ModuleList([TransformerBlock(config) for _ in range(num_blocks)])
        

    def forward(self, x: Float[torch.Tensor, "seq_len vocab"]) -> Float[torch.Tensor, "vocab seq_len"]:
        x = self.embedding(x)
        for block in self.blocks:
            x = block.forward(x)
        x = (x @ self.embedding.weight).T
        return x
    

In [ ]:
class Tokenizer():
    def __init__(self, raw_data: str):
        self.raw_data = raw_data 
        self.vocab_size = None
        self.vocab = None
        self.vocab_inverse = None
        
    def process_raw_data(self, text: str, 
                        allowed_punctuation: str = "-.,;:!?()\"" + "".join(str(x) for x in range(10)),
                        punctuation_convert: dict[str,str] = {'—': '-'}
                        ) -> str:
        for char, replacement in punctuation_convert.items():
            text = text.replace(char, replacement)
              
        text = '\n'.join(
                    line 
                    for line in text.split('\n')
                    if '.jpg' not in line
                )
        
        text = unicodedata.normalize('NFKD', text)

        # Encode to ASCII bytes, then decode back to string, ignoring errors
        text = text.encode('ascii', 'ignore').decode('ascii')

        # remove newlines and tabs
        text = text.replace('\n', ' ').replace('\t', ' ')

        for char in allowed_punctuation:
            text = text.replace(char, f' {char} ')
              
        text = text.strip()

        # remove multiple spaces
        while '  ' in text:
            text = text.replace('  ', ' ')

        text = ''.join((char if (char.isalnum() or char in allowed_punctuation or char == ' ') else ' ') for char in text)
        
        text = text.lower()

        text = text.strip()

        return text
    
    def tokenize(self, 
        text: str,
        process: bool = False,
    ) -> list[int]:
        if process:
            text = self.process_raw_data(text)
        tokenized_text = text.split(' ')

        vocab_counts: Counter[str] = Counter(tokenized_text).most_common()
        self.vocab_size: int = len(vocab_counts)
        self.vocab = [token for token, count in vocab_counts]
        self.vocab_inverse = {key: value for key, value in (range(0, self.vocab_size), self.vocab)}

# Tests

In [9]:
# Attention head test
x: Float[torch.Tensor, "seq_len d_model"] = torch.ones(5, 16)
config = Config(d_model=16, d_vocab=1000, d_hidden=64)
attention_head: AttentionHead = AttentionHead(config)
output: Float[torch.Tensor, "seq_len d_model"] = attention_head.forward(x)
print(output.shape)

torch.Size([5, 16])


In [10]:
# Test the whole thing
config = Config(d_model=16, d_vocab=1000, d_hidden=64)
transformer = Transformer(num_blocks=2, config=config)
x = torch.ones(config.d_vocab, dtype=torch.float)
y: Float[torch.Tensor, "vocab seq_len"] = transformer(x)
print(y.shape)
print(y)
print(x)

torch.Size([1000])
tensor([-3.0392e-02,  2.3679e-02,  4.2455e-03,  5.7964e-02,  1.8441e-02,
         2.1536e-02,  4.0499e-03,  1.9642e-02,  4.8857e-02, -6.3464e-02,
         6.5552e-03,  6.5082e-03, -1.8573e-02,  3.5810e-02,  4.8736e-02,
        -7.7834e-02,  5.0728e-02,  3.0764e-02, -2.5055e-03,  1.9068e-02,
        -3.6373e-03,  1.0339e-02,  1.5741e-02,  4.3782e-02, -2.3166e-02,
        -3.5456e-02,  8.8922e-03,  2.3663e-02, -1.5116e-02, -4.0177e-03,
        -2.5768e-02,  3.4168e-02, -1.0990e-02,  3.1342e-02, -9.2257e-03,
         2.9073e-02, -3.3552e-02, -1.0607e-04,  4.8377e-03,  4.9110e-02,
         1.7104e-03, -3.2923e-02, -1.0801e-02,  7.0406e-03, -7.8230e-03,
         1.0830e-02,  2.8227e-02, -2.5068e-03,  6.6414e-03,  2.8692e-02,
         8.0468e-03, -3.7961e-04, -8.3236e-03, -3.1042e-02, -4.7181e-02,
        -7.2676e-04, -1.8590e-02,  1.3625e-02, -3.6877e-02,  3.2727e-02,
         4.2861e-02, -3.6292e-02,  6.1446e-03,  3.9727e-02,  1.9036e-02,
        -5.3235e-02,  5.4263e-02

C:\Users\eteco\AppData\Local\Temp\ipykernel_16952\1079021095.py:13: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3687.)
  x = (x @ self.embedding.weight).T


# Training Loop

In [ ]:
def train_model(
    model: Transformer,
    loss: torch.nn.CrossEntropyLoss = nn.CrossEntropyLoss(),
    lr: Float = 1e-3,
    epochs: Int = 1
    ):
    optimizer: torch.optim.SGD = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    string_data = get_many_books(ids=range(10,15), data_temp="./data/gutenberg_data")
    tokenized_string_data = [x for i in string_data[]]
    labels = {}
    int_labels = []
    for i in range(len(string_data)):
        labels[i] = string_data[i]
        int_labels.append(i)
    
    training_data = torch.tensor(int_labels)
    
    for epoch in range(epochs):
        
        optimizer.zero_grad()
        outputs = model(training_data)
    



In [46]:
config: Config = Config(d_model=16, d_vocab=1000, d_hidden=64)
model = Transformer(num_blocks=2, config=config)
train_model(model)

Getting book 10...
	4432261 characters read
Getting book 11...
	148062 characters read
Getting book 12...
	168390 characters read
Getting book 13...
	34579 characters read
Getting book 14...
	1951150 characters read


RuntimeError: mat1 and mat2 must have the same dtype, but got Long and Float